<a href="https://colab.research.google.com/github/webjjang/data_anal/blob/main/ch03_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 필요한 파일 다운 받기
import gdown
gdown.download("https://bit.ly/3RhoNho", "ns_202104.csv", quiet=False)


Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:00<00:00, 112MB/s]


'ns_202104.csv'

In [3]:
# pandas로 데이터 일어오기
import pandas as pd
ns_df = pd.read_csv("ns_202104.csv", low_memory=False)
# 13 의미없는 열 제거
ns_book = ns_df.loc[:, "번호":"등록일자"]

In [7]:
selected_rows = ns_df["출판사"] == "한빛미디어"
ns_book2 = ns_book[selected_rows]
count_df = ns_book[["도서명", "저자", "ISBN", "권", "대출건수"]]
group_df = count_df.groupby(by=["도서명", "저자", "ISBN", "권"], dropna=False)
loan_count = group_df.sum()
# 도서 대여가 중복이 되는 데이터
# 처음 데이터는 중복이 되지 않았다. - False, 2번 데이터부터는 중복이 되었다. - True
dup_rows = ns_book.duplicated(subset=["도서명", "저자", "ISBN", "권"])
# 중복이 안되는 데이터
unique_rows = ~dup_rows
ns_book3 = ns_book[unique_rows].copy()
ns_book3.set_index(["도서명", "저자", "ISBN", "권"], inplace=True)
ns_book3.update(loan_count)
ns_book4 = ns_book3.reset_index()
ns_book4.head()

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈 지음,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형 지음,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,9788934990833,NaN,5,김영사,2021,NaN,NaN,NaN,1,0,2021-03-19


In [14]:
# 정규 표현식을 이용한 데이터 수정
# replace({"키:항목":{키 찾는데이타:바꿀데이터}})
# 찾는 데이터를 정규 표현식 활용한 바꾸기
# 일반적인 replace
# ns_book4.replace({"발행년도":{"2021":"21"}})[100:102]
# 정규 표현식을 이용한 replce : 한번에 yyyy -> yy 형식으로 바꾼다. r"문자열" : 문자열이 정규 표현식
# 정규 표현식의 \d -> 숫자 데이터, () -> 그룹으로 만듬.
# r"\1" - 첫번재 그룹의 데이터
# ns_book4.replace({"발행년도":{r"\d\d(\d\d)": r"\1"}}, regex=True)[100:102]
# 정규표현식 문자/문자열의 반복 : 바로 뒤에 {} - {2} - 2번 반복되어야 한다. {2,4} - 2번부터 4번 반복한다.
ns_book4.replace({"발행년도":{r"\d{2}(\d{2})": r"\1"}}, regex=True)[100:102]


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",9788958743019,NaN,101,좋은씨앗,18,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",9788960518483,NaN,102,부키,21,NaN,NaN,325.1,1,0,2021-03-15


In [25]:
# 저장 항목에서 (지은이), (옮긴이) 제거해 보자
# () - 그룹으로 처리, \(\) - 순수 문자열 ()로 처리
# . - 모든 문자, \s - 공백문자
# (.*) - 지은이 앞에 사람이름 : 첫번째 그룹 - \1, (.*) - 옮긴이 앞에 사람이름 + 앞에 "," + "blk" : 두번째 그룸 - \2
ns_book4.replace({"저자":{r"(.*)\(지은이\)(.*)\s\(옮긴이\)": r"\1\2"}}, regex=True)[100:102]
# ns_book4.replace({"저자":{r"\(옮긴이\)": ""}}, regex=True, inplace=True)
# ns_book4.head()
# ns_book4[100:102]

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 , 김진웅",9788958743019,NaN,101,좋은씨앗,2018,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 , 안기순",9788960518483,NaN,102,부키,2021,NaN,NaN,325.1,1,0,2021-03-15


In [21]:
ns_book4.replace({"저자":{r"(.*)\s지음(.*)\s옮김": r"\1\2"}}, regex=True).replace({"저자":{r"(.*)\s지음": r"\1"}}, regex=True)[:4]


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜, 임재희",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19


In [26]:
# 발행년도를 숫자로 변경해보자 -> 숫자가 아닌 데이터가 포함되어 있다.
ns_book4.astype({"발행년도":"int64"})

ValueError: invalid literal for int() with base 10: '1988.': Error while type casting for column '발행년도'

In [27]:
# 데이터가 1998이 포함이 되어 있는 데이터 찾아서 갯수를 합해줘라.
ns_book4["발행년도"].str.contains("1988").sum()

407

In [37]:
# contains(str=정규표현식 | r"" -> r을 생략해도 된다.
# 발행년도에서 숫자로 변경 불가능한 데이터 확인 - \D : 숫자가 아닌 거, \d - 숫자인거
invalid_number = ns_book4["발행년도"].str.contains("\D", na=True)
print(invalid_number.sum())
ns_book4[invalid_number].head()

1777


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
19138,단국강토,홍태수 저,9788974420031,NaN,19565,매일경제신문사,1988.,NaN,NaN,511.1,1,1,2019-12-19
19227,삼성의 역사,송부웅 撰,9788985464369,NaN,19736,삼양,단기4334[2001],NaN,0,911.02,1,1,2019-12-06
26097,배고픈 애벌레,에릭 칼 글·그림 ;이희재 옮김,9788959514083,NaN,26812,더큰컴퍼니,[2019],NaN,NaN,843,1,0,2019-08-12
29817,(The) Sopranos sessions,"Matt Zoller Seitz,$eAlan Sepinwall,$eLaura Lip...",9781419734946,NaN,30586,Harry N Abrams Inc,2019.,NaN,NaN,326.76,1,0,2019-06-13
29940,다음엔 너야,에른스트 얀들 글;노르만 융에 그림;박상순 옮김,9788949110646,NaN,30709,비룡소,2018.,9788949110004,7,853,1,9,2019-06-04


In [44]:
# 단기4334[2001] 데이터로 정규표현식 r".*(\d{4}).*" 처리를 하면 2001이 나온다. 탐욕적 검색으로 마지막 매칭까지 찾아가서
# 현재 위치 마지막 숫자인 2001이된다.
ns_book5 = ns_book4.replace({"발행년도" : r".*(\d{4}).*"}, r"\1", regex=True)
ns_book5[invalid_number].head()

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
19138,단국강토,홍태수 저,9788974420031,NaN,19565,매일경제신문사,1988,NaN,NaN,511.1,1,1,2019-12-19
19227,삼성의 역사,송부웅 撰,9788985464369,NaN,19736,삼양,2001,NaN,0,911.02,1,1,2019-12-06
26097,배고픈 애벌레,에릭 칼 글·그림 ;이희재 옮김,9788959514083,NaN,26812,더큰컴퍼니,2019,NaN,NaN,843,1,0,2019-08-12
29817,(The) Sopranos sessions,"Matt Zoller Seitz,$eAlan Sepinwall,$eLaura Lip...",9781419734946,NaN,30586,Harry N Abrams Inc,2019,NaN,NaN,326.76,1,0,2019-06-13
29940,다음엔 너야,에른스트 얀들 글;노르만 융에 그림;박상순 옮김,9788949110646,NaN,30709,비룡소,2018,9788949110004,7,853,1,9,2019-06-04


In [45]:
# 발행년도가 숫자가 아닌 데이터 확인
unknown_year = ns_book5["발행년도"].str.contains("\D", na=True)
print(unknown_year.sum())
ns_book5[unknown_year].head()

67


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
30838,본격 한중일 세계사 5,굽시니스트 지음,9791190065092,NaN,31616,위즈덤하우스,NaN,NaN,NaN,NaN,0,0,2019-05-28
39130,정책금융의 현황과 발전과제,정책금융연구회,9788992784108,NaN,40141,한국산업은행,NaN,NaN,NaN,327.1,1,0,2019-01-22
39256,서울지역 유적 발굴조사 총서 3,서울역사박물관,9791186324714,NaN,40268,서울역사박물관,NaN,9791186324431,NaN,NaN,0,0,2019-01-22
76836,흰머리 큰줄기,한호진 지음,9788973010769,NaN,81202,秀文出版社,[발행년불명],NaN,0,699.1,1,1,2016-11-10
150543,(속) 경제학사,박장환 지음,9788994339207,NaN,160436,NaN,[20--],NaN,1,320.9,1,2,2012-11-19


In [46]:
# 숫자가 아닌 데이터의 처리
# 숫자가 아닌 데이터에 "-1"로 세팅한다. 그리고 int 로 변경 다시 저장해서 원본 데이터 변경시키자.
ns_book5.loc[unknown_year, "발행년도"] = "-1"
ns_book5 = ns_book5.astype({"발행년도":"int64"})
# gt() > 4000 -> grater then : 단기 4334 데이터만 존재해서 숫자 데이터로 잡혔다.
# lt() < 4000 : 작은 less then, eq() == 4000 : 같은 equal
ns_book5["발행년도"].lt(4000).sum()

np.int64(384460)

In [47]:
# 4000 보다 큰 경우 2333을 빼서 서기로 바뀐다. 처리해 줘야한다. 단기 : 단군 시대의 년도
dangun_yy_rows = ns_book5["발행년도"].gt(4000)
# 단기를 서기로 변환해서 다시 넣는다.
ns_book5.loc[dangun_yy_rows, "발행년도"]  = ns_book5.loc[dangun_yy_rows, "발행년도"] - 2333



In [49]:
# 교재에서는 4000보다 큰 처리를 했다. 현재 년도 보다 큰 : 2025 : -1로 변경해 준다.
dangun_yy = ns_book5["발행년도"].gt(2025)
print(dangun_yy.sum())
ns_book5[dangun_yy].head(2)

49


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
7768,"어딘가 상상도 못 할 곳에, 수많은 순록 떼가(움도1)","켄 리우 지음, 장성주 옮김",9791158887162,NaN,7971,황금가지,2030,NaN,NaN,NaN,1,2,2020-09-01
7865,생동하는 물질,"제인 베넷 지음, 문성재 옮김",9788965642527,NaN,8068,현실문화,2200,NaN,NaN,NaN,1,0,2020-09-01


In [54]:
# 현재 년도 2025 보다 큰 데이터를 -1로 세팅 (숫자가 아닌 데이터를 -1로 세팅 선처리함.)
ns_book5.loc[dangun_yy, "발행년도"] = "-1"
# ns_book5["발행년도"].gt(2025).sum()
ns_book5[dangun_yy].head(2)

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
7768,"어딘가 상상도 못 할 곳에, 수많은 순록 떼가(움도1)","켄 리우 지음, 장성주 옮김",9791158887162,NaN,7971,황금가지,-1,NaN,NaN,NaN,1,2,2020-09-01
7865,생동하는 물질,"제인 베넷 지음, 문성재 옮김",9788965642527,NaN,8068,현실문화,-1,NaN,NaN,NaN,1,0,2020-09-01
